# Laboratorio 2017-10-16

# Numpy
Numpy è una libreria matematica che permette di effettuare varie operazioni matematiche. Di seguito alcuni esempi d'uso:

In [1]:
import numpy as np

In [2]:
v = np.arange(10)
v

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [3]:
v = v.reshape(5,2)
v

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [4]:
v = np.linspace(0, 2, 9)
v

array([ 0.  ,  0.25,  0.5 ,  0.75,  1.  ,  1.25,  1.5 ,  1.75,  2.  ])

In [5]:
v = np.sin(v)
v

array([ 0.        ,  0.24740396,  0.47942554,  0.68163876,  0.84147098,
        0.94898462,  0.99749499,  0.98398595,  0.90929743])

In [6]:
n = np.random.uniform(0, 2*np.pi)
n

0.5210865305623954

In [7]:
v = np.random.random(10)
v

array([ 0.25404885,  0.60222739,  0.73962081,  0.92045192,  0.81511413,
        0.4514217 ,  0.99168582,  0.45649259,  0.00561888,  0.55007677])

In [8]:
v = v - np.random.random(10)
v

array([ 0.23549515, -0.37814518,  0.27325901,  0.66140807,  0.78613876,
       -0.04567506,  0.67712889,  0.34692113, -0.09971398,  0.13803715])

In [9]:
v = v.reshape(5,2)
v

array([[ 0.23549515, -0.37814518],
       [ 0.27325901,  0.66140807],
       [ 0.78613876, -0.04567506],
       [ 0.67712889,  0.34692113],
       [-0.09971398,  0.13803715]])

In [10]:
v = v.ravel()
v

array([ 0.23549515, -0.37814518,  0.27325901,  0.66140807,  0.78613876,
       -0.04567506,  0.67712889,  0.34692113, -0.09971398,  0.13803715])

## Assignment 1
Produce (and draw) 100 random points within the unit square $ [0, 1]^2 $

In [11]:
from larlib import *

random_numbers = [np.random.random(2) for i in np.arange(1000)]

VIEW(STRUCT(map(MK, random_numbers)))

Evaluating fenvs.py..
...fenvs.py imported in 0.005188 seconds


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x113f9d1e0> >

![square](images/square.jpeg)

## Assignment 2
Produce (and draw) 1000 random points within $ S_1 $, the $1D$ sphere (circle) of unit radius centered at the origin $(0, 0)$

In [12]:
def random_point_circle():
    x = np.random.uniform(-1, 1)
    y = np.random.uniform(-sqrt(1-x**2), sqrt(1-x**2))
    return x, y


VIEW(STRUCT(map(MK, [random_point_circle() for i in np.arange(10000)])))


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x117926690> >

![circle](images/circle.jpeg)

## Start using “LarLib” with simplicial complexes

### larSimplexFacets

In [13]:
def larSimplexFacets(simplices):
    out = []
    d = len(simplices[0])
    for simplex in simplices:
        out += AA(sorted)(
            [simplex[0:k]+simplex[k+1:d] for k in range(d)])
    out = set(AA(tuple)(out))
    return  sorted(out)

#### Example 1

In [14]:
simplices = [[1,2,4],[2,4,5],[2,3,5]]
larSimplexFacets(simplices)

[(1, 2), (1, 4), (2, 3), (2, 4), (2, 5), (3, 5), (4, 5)]

#### Example 2

In [15]:
V = [[0,0], [0,1],[2,0],[4,0],[1.5,3],[3.5,2.5]]
FV = simplices
model = V,FV
VIEW(STRUCT(MKPOLS(model)))
VIEW(SKEL_1(STRUCT(MKPOLS(model))))
EV = larSimplexFacets(FV)
VIEW(STRUCT(MKPOLS((V,EV))))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x113e94ea0> >

![larSimplexFacets-example1](images/larSimplexFacets-example1.jpeg)

### larExtrude1

In [16]:
def larExtrude1(model,pattern):
    """ Simplicial model extrusion in accord with a 1D pattern """
    V, FV = model
    d, m = len(FV[0]), len(pattern)
    coords = list(cumsum([0]+(AA(ABS)(pattern))))
    offset, outcells, rangelimit = len(V), [], d*m
    for cell in FV:
            tube = [v + k*offset for k in range(m+1) for v in cell]
            cellTube = [tube[k:k+d+1] for k in range(rangelimit)]
            outcells += [reshape(cellTube, newshape=(m,d,d+1)).tolist()]
    outcells = AA(CAT)(TRANS(outcells))
    cellGroups = [group for k,group in enumerate(outcells) if pattern[k]>0]
    outVertices = [v+[z] for z in coords for v in V]
    outModel = outVertices, CAT(cellGroups)
    return outModel

#### Example 1

In [17]:
V = [[0,0],[1,0],[2,0],[0,1],[1,1],[2,1],[0,2],[1,2],[2,2]]
FV = [[0,1,3],[1,2,4],[2,4,5],[3,4,6],[4,6,7],[5,7,8]]
model = larExtrude1((V,FV),4*[1,2,-3])
VIEW(EXPLODE(1,1,1.2)(MKPOLS(model)))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1171c0120> >

#### Example 2

In [18]:
model = larExtrude1( VOID, 10*[1] )
VIEW(EXPLODE(1.5,1.5,1.5)(MKPOLS(model)))
model = larExtrude1( model, 10*[1] )
VIEW(EXPLODE(1.5,1.5,1.5)(MKPOLS(model)))
model = larExtrude1( model, 10*[1] )
VIEW(EXPLODE(1.5,1.5,1.5)(MKPOLS(model)))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x113e94f90> >

#### Example 3

In [19]:
model = larExtrude1( VOID, 10*[1,-1] )
VIEW(EXPLODE(1.5,1.5,1.5)(MKPOLS(model)))
model = larExtrude1( model, 10*[1] )
VIEW(EXPLODE(1.5,1.5,1.5)(MKPOLS(model)))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x113f9d930> >

In [20]:
quit()